In [1]:
from itertools import permutations, product, combinations, combinations_with_replacement

# Ejercicio 1

a) Genere las suceciones de 4 digitos diferentes e imprima su conteo 


In [2]:
digitos = list(range(10)) # lista de digitos de 0 a 9

sin_repeticion = list(permutations(digitos, 4)) # realiza la permutacion de 4 digitos
len(sin_repeticion)

5040

b) Genere las sucesiones de 4 dígitos que contegan uno o más dígitos repetidos e imprima su conteo

In [3]:
todo = list(product(digitos, repeat = 4)) # realiza la lista con todas las suceciones
con_repeticion = set(todo) - set(sin_repeticion) #se quitan las sucesiones únicas
len(con_repeticion) #conteo de las suceciones con repeticion

4960

c) Imprima el conteo de elementos por cada una de las siguientes divisiones, utilizando la generación que realizó en el inciso b
  * 1) Se repite el dígito 4 veces

In [4]:
repetidos = list(con_repeticion) # convertir repetidos a una lista 
lista_todos_iguales = [ i for i in repetidos if (i[0] == i[1] == i[2] == i[3])] # se agrega el elemento si se parecen 
len(lista_todos_iguales)

10

* 2) Se repite dos dígitos dos veces cada uno

In [5]:
conjunto_sin_todos_iguales = set(repetidos) - set(lista_todos_iguales) # se quitan todos los que sean iguales 
sin_todos_iguales = list(conjunto_sin_todos_iguales) # se vuelve a lista 
lista_dos_iguales = [ i for i in sin_todos_iguales if (((i[0] == i[1]) and (i[2]==i[3])) or ((i[0] == i[2]) and (i[1]==i[3])) or ((i[0] == i[3]) and (i[1]==i[2])))]
len(lista_dos_iguales)

270

* 3) Se repite un elemento dos veces y los otros no se repiten

In [6]:
conjunto_sin_parejas = set(repetidos) - set(lista_todos_iguales) - set(lista_dos_iguales) # se quitan todos los que sean iguales, e iguales en pares
sin_uno_dos = list(conjunto_sin_parejas) # se vuelve a lista 
lista_un_par = [ i for i in sin_uno_dos if (((i[0] == i[1])!=(i[2]!=i[3])) or ((i[0]==i[2])!=(i[1]!=i[3])) or ((i[0] == i[3])!=(i[2]!=i[1])) or
                                           ((i[1] == i[2])!=(i[0]!=i[3]))  or ((i[2] == i[3])!=(i[0]!=i[1])))]
len(lista_un_par)

4320

* 4) Se repite un dígito 3 veces y el otro no se repite

In [7]:
conjunto_complemento = conjunto_sin_parejas - set(lista_un_par) # se quitan los incisos
sin_uno_a_tres = list(conjunto_complemento) # se vuelve a lista 
lista_tres_iguales = [i for i in sin_uno_a_tres if ((i[0]==i[1]==i[2]) or (i[0]==i[1]==i[3]) or (i[0]==i[2]==i[3]) or (i[1]==i[2]==i[3]))]
len(lista_tres_iguales)

360

# Ejercicio 2


In [8]:
def contarPrimos(n):
    x = 0
    numerosList = []
    
    for num in range(n):
        if num <= 1:
            continue
        for i in range(2, num):
            if (num % i) == 0:
                break
        else:
            numerosList.append(num)
            x += 1

    return x, numerosList

print(contarPrimos(100))


(25, [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97])


In [9]:
numeros = list(range(2,101))

dos = set([i for i in numeros if (i%2==0 and i!=2)])
tres = set([i for i in numeros if (i%3==0 and i!=3)])
cinco = set([i for i in numeros if (i%5==0 and i!=5)])
siete = set([i for i in numeros if (i%7==0 and i!=7)])


primos = set(numeros) - (dos|tres|cinco|siete) # esto es el principio de inclusión exclusión, que es la unión de los conjuntos
                                              # se realiza el complemento 
print(len(list(primos))) 
print(list(primos))

25
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


# Ejercicio 3

In [10]:
n=7
m=5 
k=0
tuplas = []
for i1 in range(1,n+1):
    for i2 in range(1,i1+1):
      for i3 in range(1,i2+1):
        for i4 in range(1, i3+1):
          for i5 in range(1, i4+1):
            k=k+1
            tuplas.append((i5, i4, i3, i2, i1))

conjunto = (set(tuplas))
print(k)
print(conjunto)



elementos = (1,2,3,4,5,6,7)
muestra = (list(combinations_with_replacement(elementos,5)))

print(muestra)

462
{(2, 3, 3, 3, 7), (1, 2, 2, 2, 6), (4, 6, 6, 7, 7), (1, 3, 3, 4, 7), (1, 5, 6, 7, 7), (2, 3, 4, 4, 5), (2, 2, 3, 5, 7), (5, 5, 5, 5, 6), (1, 1, 3, 3, 5), (1, 2, 2, 5, 7), (1, 2, 7, 7, 7), (3, 4, 5, 6, 6), (1, 1, 3, 4, 6), (3, 3, 3, 3, 5), (3, 6, 6, 6, 6), (2, 5, 5, 7, 7), (3, 4, 5, 7, 7), (2, 4, 6, 6, 6), (1, 3, 5, 7, 7), (2, 2, 2, 5, 7), (1, 2, 3, 5, 6), (1, 1, 1, 2, 7), (1, 5, 6, 6, 6), (3, 3, 5, 5, 7), (3, 4, 4, 7, 7), (2, 2, 3, 3, 5), (1, 1, 2, 4, 5), (1, 4, 4, 4, 7), (1, 1, 4, 4, 7), (2, 3, 7, 7, 7), (2, 2, 3, 4, 6), (3, 3, 7, 7, 7), (4, 4, 5, 6, 7), (1, 3, 3, 4, 6), (2, 3, 3, 5, 6), (1, 1, 1, 2, 2), (2, 2, 3, 7, 7), (2, 2, 5, 5, 7), (1, 1, 1, 5, 7), (6, 6, 6, 6, 7), (2, 6, 6, 6, 6), (1, 1, 6, 7, 7), (1, 1, 3, 3, 4), (2, 3, 3, 6, 7), (1, 2, 2, 5, 6), (3, 5, 5, 6, 6), (4, 4, 4, 5, 7), (2, 6, 6, 7, 7), (3, 3, 6, 7, 7), (3, 3, 3, 3, 4), (2, 2, 2, 4, 5), (1, 3, 4, 4, 5), (3, 5, 5, 7, 7), (1, 2, 3, 4, 4), (2, 2, 4, 5, 7), (2, 2, 2, 5, 6), (1, 6, 7, 7, 7), (1, 1, 1, 1, 5), (1, 2, 6,

Al comparar el conjunto de tuplas generada por el anidamiento del ciclo for con las tuplas generadas con la funcion de combinaciones con remplazo de intertools, se puede observar algunas diferencias. La primera es que al utilizar las combinaciones con remplazo se observa que las tuplas tienen un orden logico de combinaciones más reconocible , ya que este va desde todas las combinaciones del 1 hasta las del 5, mientras que en las otra tuplas no es asi. También el uso de anidación ocasiona que las lineas de codigo sean mucho más en comparación al uso de herramientas, dando la misma cantidad de combinaciones con las muestras y elementos totales seleccionados.






